In [6]:
!pip install -U ydata-profiling
!pip install --upgrade jupyter ipywidgets

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 17.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.9/254.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/4

In [22]:
import pandas as pd

df1 = pd.read_excel('/home/aistudio/data/data237885/附件1.xlsx')
df2 = pd.read_excel('/home/aistudio/data/data237885/附件2.xlsx')
df3 = pd.read_excel('/home/aistudio/data/data237885/附件3.xlsx')
df4 = pd.read_excel('/home/aistudio/data/data237885/附件4.xlsx')

In [53]:
# 使用单品编码进行合并，并且只保留附件2中已有的单品编码
merged_df = pd.merge(df2, df1[['单品编码', '单品名称', '分类名称']], on='单品编码', how='left')

# 转为日期格式
merged_df['销售日期'] = pd.to_datetime(merged_df['销售日期'])

# 新增一个表示所在周的列
merged_df['周'] = merged_df['销售日期'].dt.to_period('W')

# 新增一个表示所在月份的列
merged_df['月份'] = merged_df['销售日期'].dt.to_period('M')

# 创建一个新的列"季度"
merged_df['季度'] = merged_df['销售日期'].dt.to_period('Q')

# 将销售日期转换为字符串格式，以保持原来的格式（只包含日期部分）
merged_df['销售日期'] = merged_df['销售日期'].dt.strftime('%Y-%m-%d')

# 将合并后的DataFrame保存为新的Excel文件
merged_df.to_excel("/home/aistudio/data/data237885/merged_attachment.xlsx", index=False)

print(merged_df.head())

         销售日期        扫码销售时间             单品编码  销量(千克)  销售单价(元/千克) 销售类型 是否打折销售  \
0  2020-07-01  09:15:07.924  102900005117056   0.396         7.6   销售      否   
1  2020-07-01  09:17:27.295  102900005115960   0.849         3.2   销售      否   
2  2020-07-01  09:17:33.905  102900005117056   0.409         7.6   销售      否   
3  2020-07-01  09:19:45.450  102900005115823   0.421        10.0   销售      否   
4  2020-07-01  09:20:23.686  102900005115908   0.539         8.0   销售      否   

      单品名称 分类名称                      周       月份      季度  
0  泡泡椒(精品)  辣椒类  2020-06-29/2020-07-05  2020-07  2020Q3  
1      大白菜  花叶类  2020-06-29/2020-07-05  2020-07  2020Q3  
2  泡泡椒(精品)  辣椒类  2020-06-29/2020-07-05  2020-07  2020Q3  
3      上海青  花叶类  2020-06-29/2020-07-05  2020-07  2020Q3  
4       菜心  花叶类  2020-06-29/2020-07-05  2020-07  2020Q3  


In [54]:
# 检查“销售类型”是“销售”，但“销量(千克)”不是正数的情况
mask1 = (merged_df['销售类型'] == '销售') & (merged_df['销量(千克)'] <= 0)
# 检查“销售类型”是“退货”，但“销量(千克)”不是负数的情况
mask2 = (merged_df['销售类型'] == '退货') & (merged_df['销量(千克)'] >= 0)

# 将不符合条件的行选出来
invalid_rows1 = merged_df[mask1]
invalid_rows2 = merged_df[mask2]

if len(invalid_rows1) > 0:
    print("以下行的‘销售类型’是‘销售’，但‘销量(千克)’不是正数：")
    print(invalid_rows1)
if len(invalid_rows2) > 0:
    print("以下行的‘销售类型’是‘退货’，但‘销量(千克)’不是负数：")
    print(invalid_rows2)

if len(invalid_rows1) == 0 and len(invalid_rows2) == 0:
    print("所有数据都满足条件。")

所有数据都满足条件。


In [55]:
# 按季度分组，并计算每个季度的销售总量
sales_by_quarter = merged_df.groupby('季度')['销量(千克)'].sum().reset_index()

# 将结果写入Excel文件
sales_by_quarter.to_excel('/home/aistudio/data/data237885/sales_by_quarter.xlsx', index=False)

In [56]:
# 按季度和分类名称分组，并计算每个组的销售总量
sales_by_quarter_and_category = merged_df.groupby(['季度', '分类名称'])['销量(千克)'].sum().reset_index()

sales_by_quarter_and_category.to_excel('/home/aistudio/data/data237885/sales_by_quarter_and_category.xlsx', index=False)

In [57]:
# 按销售日期和分类名称分组，并计算每一天的各类销量之和
sales_by_day_and_category = merged_df.groupby(['销售日期','分类名称'])['销量(千克)'].sum().reset_index()

sales_by_day_and_category.to_excel('/home/aistudio/data/data237885/sales_by_day_and_category.xlsx',index=False)

In [58]:
# 按月份和分类名称分组，计算每一月的各类销售量之和
sales_by_month_and_category = merged_df.groupby(['月份','分类名称'])['销量(千克)'].sum().reset_index()

sales_by_month_and_category.to_excel('/home/aistudio/data/data237885/sales_by_month_and_category.xlsx',index=False)

In [59]:
# 按周和分类名称分组，计算每一周的各类销售量之和
sales_by_week_and_category = merged_df.groupby(['周','分类名称'])['销量(千克)'].sum().reset_index()

sales_by_week_and_category.to_excel('/home/aistudio/data/data237885/sales_by_week_and_category.xlsx',index=False)

In [39]:
merged_df.describe()

,单品编码,销量(千克),销售单价(元/千克)
count,8.785030e+05,878503.000000,878503.000000
mean,1.030313e+14,0.536112,8.917144
std,7.174368e+11,0.400087,6.311265
min,1.029000e+14,-9.082000,0.100000
25%,1.029000e+14,0.290000,4.900000
50%,1.029000e+14,0.435000,7.900000
75%,1.029000e+14,0.742000,10.800000
max,1.069740e+14,160.000000,119.900000


In [34]:
import numpy as np
from ydata_profiling import ProfileReport

# 创建数据分析报告
profile = ProfileReport(merged_df, title='Your Data Analysis Report', explorative=True)

# 保存报告为HTML文件
profile.to_file("/home/aistudio/data/data237885/merged_ydata_report.html")

# 显示报告（适用于Jupyter Notebook）
profile.to_notebook_iframe()

Summarize dataset:  50%|█████     | 13/26 [02:21<03:13, 14.91s/it, scatter 单品编码, 单品编码]            /opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 21333 (\N{CJK UNIFIED IDEOGRAPH-5355}) missing from current font.
  plt.savefig(
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 21697 (\N{CJK UNIFIED IDEOGRAPH-54C1}) missing from current font.
  plt.savefig(
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 32534 (\N{CJK UNIFIED IDEOGRAPH-7F16}) missing from current font.
  plt.savefig(
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/ydata_profiling/visualisation/utils.py:73: UserWarning: Glyph 30721 (\N{CJK UNIFIED IDEOGRAPH-7801}) missing from current font.
  plt.savefig(
Summarize dataset:  54%|█████▍    | 14/26 [02:21<02:15, 11

In [38]:
from datetime import datetime

# 定义数据中最早和最晚的两个日期字符串
date_str1 = '2020-07-01 00:00:00'
date_str2 = '2023-06-30 00:00:00'

# 将日期字符串转换为 datetime 对象
date1 = datetime.strptime(date_str1, '%Y-%m-%d %H:%M:%S')
date2 = datetime.strptime(date_str2, '%Y-%m-%d %H:%M:%S')

# 计算两个日期之间的天数差
delta = date2 - date1

# 输出天数
print(f"间隔天数: {delta.days}（天）")


间隔天数: 1094（天）
